In [1]:
!pip install xgboost shap utils


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
#Imports
import pandas as pd

import matplotlib.pyplot as plt

import xgboost as xgb
from xgboost import plot_importance

import shap
shap.initjs()

import sys
sys.path.append('../')
import utils

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
#Load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/fenago/datasets/refs/heads/main/insurance.csv")

print("Number of rows: {}".format(len(data)))
data.head(10)

In [ ]:
#Feature engineering
data['sex'] = [1 if s =='male' else 0 for s in data['sex']]
data['smoker'] = [1 if s =='yes' else 0 for s in data['smoker']]

data['region.SW'] = [1 if r =='southwest' else 0 for r in data['region']]
data['region.SE'] = [1 if r =='southeast' else 0 for r in data['region']]
data['region.NW'] = [1 if r =='northwest' else 0 for r in data['region']]
data['region.NE'] = [1 if r =='northeast' else 0 for r in data['region']]

data = data.drop('region', axis=1)

data.head(10)

In [ ]:
y = data['charges']
X = data.drop('charges', axis=1)

#Train model
model = xgb.XGBRegressor(objective="reg:squarederror"
                         ,n_estimators=10) 
model.fit(X, y)

#Get predictions
y_pred = model.predict(X)

#Model evaluation
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,8))

plt.scatter(y,y_pred)
plt.plot([0, 60000], [0, 60000], color='r', linestyle='-', linewidth=2)

plt.ylabel('Predicted',size=20)
plt.xlabel('Actual',size=20)